In [1]:
from pathlib import Path
import sys
import json
import shutil

# Поднимаемся на два уровня вверх (из notebooks/ в InteriorClass/)
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))  # Теперь Python увидит src/

from src.config import RANDOM_SEED, SPLIT_RATIO, MIN_VAL_TEST_PER_CLASS, CLASS_LABELS
from src.dataset.splitter import DatasetSplitter
from src.dataset.interior_dataset import InteriorDataset, get_transforms
from src.models.interior_classifier_EfficientNet_B3 import InteriorClassifier
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torchvision.transforms as transforms
from PIL import Image

In [2]:
# 1. Собираем все пути
current_dir = Path.cwd()
root_project = current_dir.parent.parent
data_dir = root_project / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"
cian_data_dir = data_dir / "cian_data"
exp_dir = root_project / Path("experiments/exp001_baseline/results")
checkpoint_path = exp_dir / "best_model.pth"

data_dir: /home/little-garden/CodeProjects/InteriorClass/data


In [3]:
def prepare_image(image_path, img_size=(224, 224)):
    # Трансформы должны быть такими же, как при обучении!
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img = Image.open(image_path).convert("RGB")  # Обязательно конвертируем в RGB
    return transform(img).unsqueeze(0)  # Добавляем batch-размерность

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InteriorClassifier(num_classes=len(InteriorDataset.CLASSES)).to(device)

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

# Применение модели для полуавтоматической разметки на оставшемся датасете циана
batch_size = 64
model.eval()
for folder in sorted(cian_data_dir.iterdir()):
    with torch.no_grad():  # Отключаем вычисление градиентов
        batch = []
        image_paths = []  # Сохраняем пути к изображениям для текущего батча
        for image_path in tqdm(sorted(folder.iterdir()), desc=f"Processing {folder.name}"):
            
            if not image_path.is_file():
                continue

            # Подготавливаем изображение и добавляем в батч
            # Получаем изображение в формате [1, C, H, W]
            image = prepare_image(image_path).to(device)
            batch.append(image)
            image_paths.append(image_path)
            
            # Когда набрали полный батч - обрабатываем
            if len(batch) >= batch_size:
                # Объединяем все тензоры [1, C, H, W] в один [B, C, H, W]
                batch_tensor = torch.cat(batch, dim=0)

                # Получаем предсказания
                outputs = model(batch_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                confidences, class_idxs = torch.max(probabilities, dim=1)
                
                # Обрабатываем каждый элемент батча
                for img_path, confidence, class_idx in zip(image_paths, confidences, class_idxs):
                    confidence = confidence.item()
                    class_idx = class_idx.item()
                    
                    if confidence >= 0.85:
                        class_label = InteriorDataset.CLASSES[class_idx]
                        target_dir = folder / f"{class_label}_conf085"
                        target_dir.mkdir(exist_ok=True)
                        
                        # Формируем целевой путь
                        target_path = target_dir / img_path.name
                        
                        # Перемещаем файл
                        shutil.move(str(img_path), str(target_path))
                
                # Очищаем батч для следующих изображений
                batch = []
                image_paths = []
        
        # Обработка оставшихся изображений (неполный батч)
        if batch:
            # Объединяем все тензоры [1, C, H, W] в один [B, C, H, W]
            batch_tensor = torch.cat(batch, dim=0)

            outputs = model(batch_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidences, class_idxs = torch.max(probabilities, dim=1)
            
            for img_path, confidence, class_idx in zip(image_paths, confidences, class_idxs):
                confidence = confidence.item()
                class_idx = class_idx.item()
                
                if confidence >= 0.85:
                    class_label = InteriorDataset.CLASSES[class_idx]
                    target_dir = folder / f"{class_label}_conf085"
                    target_dir.mkdir(exist_ok=True)
                    
                    target_path = target_dir / img_path.name
                    shutil.move(str(img_path), str(target_path))

Processing D1_uncertain: 100%|██████████| 92141/92141 [04:12<00:00, 364.90it/s]
